<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузим-данные-и-изучим-их:" data-toc-modified-id="Загрузим-данные-и-изучим-их:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><strong>Загрузим данные и изучим их:</strong></a></span></li><li><span><a href="#Произведем-предобработку-данных:" data-toc-modified-id="Произведем-предобработку-данных:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span><strong>Произведем предобработку данных:</strong></a></span></li><li><span><a href="#Разделение-на-выборки:" data-toc-modified-id="Разделение-на-выборки:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span><strong>Разделение на выборки:</strong></a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия:" data-toc-modified-id="Логистическая-регрессия:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия:</a></span></li><li><span><a href="#Дерево-решений:" data-toc-modified-id="Дерево-решений:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений:</a></span></li><li><span><a href="#CatBoostClassifier:" data-toc-modified-id="CatBoostClassifier:-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier:</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle

from catboost import CatBoostClassifier

from tqdm.notebook import tqdm
tqdm.pandas()
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### **Загрузим данные и изучим их:**

In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
# Изучаю данные
display(data.head(10))
display(data.shape)
display(data.info())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


(159292, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

In [ ]:
# Проверяю датасет на пропуски
print('Пропусков в базе данных:\n', data.isna().sum())

Пропусков в базе данных:
 Unnamed: 0    0
text          0
toxic         0
dtype: int64


In [ ]:
# Проверяю датасет на дубликаты
print('Дубликатов в базе данных:', data.duplicated().sum())

Дубликатов в базе данных: 0


### **Произведем предобработку данных:**

In [ ]:
# Для начала удаляю столбец Unnamed, т.к. он не требуется для дальнейшей работы:
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
# Создаю функцию обработки текстов
stop_words = stopwords.words('english')
stop_words.remove('not')
lemmatizer = WordNetLemmatizer()

# Вспомогательная функция для преобразования тега части речи измодуля nltk.pos_tag в формат,
# который можно передать в WordNetLemmatizer
def get_wordnet_pos(word):
    # Преобразование тегов частей речи из pos_tag в теги WordNet
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }

    return tag_dict.get(tag, wordnet.NOUN) # Если тег не найден, используется существительное (NOUN) по умолчанию

def clean_text(text):
    # Очистка
    text =  re.sub('[^A-Za-z0-9]+', ' ', text) # оставляем только слова
    # Приведение сообщений к нижнему регистру
    text = text.lower()
    # Токенизация
    tokens = nltk.word_tokenize(text) # конвертируем тексты в токены
    # Очистка от стоп слов и лемматизация
    text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens if word not in stop_words]
    # Объединение слов в обработанных текстах
    text = ' '.join(text)

    return text

In [ ]:
# Применяю функцию clear_text для очистки
#data['text'] = data['text'].apply(clean_text)

data['text'] = data['text'].progress_apply(clean_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [ ]:
# Проверяю полученный результат
display(data.head(10))

,text,toxic
0,explanation edits make username hardcore metal...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really not try edit war guy constantly...,0
3,make real suggestion improvement wonder sectio...,0
4,sir hero chance remember page,0
5,congratulation well use tool well talk,0
6,cocksucker piss around work,1
7,vandalism matt shirvington article revert plea...,0
8,sorry word nonsense offensive anyway not inten...,0
9,alignment subject contrary dulithgow,0


### **Разделение на выборки:**

In [ ]:
# Разделяю выборки в соотношении 80/10/10:
features = data.drop(['toxic'], axis=1)
target = data['toxic']

#features_train, features_valid, target_train, target_valid = train_test_split(features,
#                                                                              target,
#                                                                              test_size=.2,
#                                                                              random_state=12345)

#features_valid, features_test, target_valid, target_test = train_test_split(features_valid,
#                                                                            target_valid,
#                                                                            test_size=.5,
#                                                                            random_state=12345)

features_train, features_test, target_train, target_test = train_test_split(features,
                                                                              target,
                                                                              test_size=.1,
                                                                              random_state=12345)


In [ ]:
# Размеры выборок:
#for i in [features_train, target_train, features_valid, target_valid, features_test, target_test]:
for i in [features_train, target_train, features_test, target_test]:
    print(i.shape)

(143362, 1)
(143362,)
(15930, 1)
(15930,)


In [ ]:
# Изучаю соотношение 1/0 в выборках на примере target_train:
toxic_1 = [i for i,x in enumerate(target_train) if x == 1]

toxic_0 = [i for i,x in enumerate(target_train) if x == 0]

toxic_bal = "{:.3f}".format(len(toxic_1) / (len(toxic_1) + len(toxic_0)))

print('Доля значений 1 для колонки \'toxic\' в тренировочной выборке:', toxic_bal)

Доля значений 1 для колонки 'toxic' в тренировочной выборке: 0.102


**Вывод:**

Имеется датасет с комментариями и их классификацией (1 - позитивный комментарий, 0 - негативный комментарий). Датасет состоит из 159292 строк и 3 столбцов.

Один из столбцов фактически является нумерацией строк начиная с 0, для дальнейшей работы не потребуется
- столбец был удален

В базе данных отсутствуют пропуски и дубликаты

Представленные в базе данных комментарии на английском языке. Присутсвует различный регистр
- текст был приведен к единому регистру

Тексты комментариев были очищены от стоп-слов, убраны лишние знаки (все за исключением букв английского алфавита) и лемматизированы

Произвел разделение на следующие выборки:
- тренировочная (80%)
- тестовая (10%)
- валидационная (10%)

Для тестовой выборки сбалансировал данные для целевого признака

## Обучение

In [ ]:
features_train = features_train.text

### Логистическая регрессия:

In [ ]:
%%time

# Обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english', sublinear_tf=True)),
                     ("lr", LogisticRegression())])

parameters = {'lr__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
              'lr__C': (.1, 1, 5, 10),
              'lr__random_state': ([12345]),
              'lr__max_iter': ([200]),}


gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

#mts = gscv.cv_results_['mean_test_score']
#lr_train_f1 = max(mts)

lr_train_f1 = gscv.best_score_

print('F1 логистической регрессии =', round(lr_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

# Валидация:
#predictions_valid = gscv.predict(features_valid.text)
#lr_valid_f1 = f1_score(target_valid, predictions_valid)
#print('F1 логистической регрессии на валидации =', round(lr_valid_f1,2))
#print()

# Тестирование:
#predictions_test = gscv.predict(features_test.text)
#lr_test_f1 = f1_score(target_test, predictions_test)
#print('финальный F1 логистической регрессии =', round(lr_test_f1,2))

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

F1 логистической регрессии = 0.77
при параметрах {'lr__C': 10, 'lr__max_iter': 200, 'lr__random_state': 12345, 'lr__solver': 'saga'}

CPU times: user 13min 37s, sys: 10min 31s, total: 24min 8s
Wall time: 24min 16s


### Дерево решений:

In [ ]:
%%time

# Обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')),
                     ("dtc", DecisionTreeClassifier())])

parameters = {'dtc__max_depth': ([x for x in range(1, 25)]),
              'dtc__random_state': ([12345]), }

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

#mts = gscv.cv_results_['mean_test_score']
#dtc_train_f1 = max(mts)

dtc_train_f1 = gscv.best_score_

print('F1 дерева решений =', round(dtc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

# Валидация:
#predictions_valid = gscv.predict(features_valid.text)
#dtc_valid_f1 = f1_score(target_valid, predictions_valid)
#print('F1 дерева решений на валидации =', round(dtc_valid_f1,2))
#print()

# Тестирование:
#predictions_test = gscv.predict(features_test.text)
#dtc_test_f1 = f1_score(target_test, predictions_test)
#print('финальный F1 дерева решений =', round(dtc_test_f1,2))

F1 дерева решений = 0.67
при параметрах {'dtc__max_depth': 24, 'dtc__random_state': 12345}

CPU times: user 14min 14s, sys: 9.66 s, total: 14min 23s
Wall time: 14min 44s


### CatBoostClassifier:

In [ ]:
%%time

# Обучение:
pipeline = Pipeline([("vect", TfidfVectorizer(stop_words='english')),
                     ("cbc", CatBoostClassifier())])

parameters = {'cbc__verbose': ([False]),
              'cbc__iterations': ([200]),}

gscv = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv.fit(features_train, target_train)

#mts = gscv.cv_results_['mean_test_score']
#cbc_train_f1 = max(mts)

cbc_train_f1 = gscv.best_score_

print('F1 CatBoostClassifier =', round(cbc_train_f1,2))
print('при параметрах', gscv.best_params_)
print()

# Валидация:
#predictions_valid = gscv.predict(features_valid.text)
#cbc_valid_f1 = f1_score(target_valid, predictions_valid)
#print('F1 CatBoostClassifier на валидации =', round(cbc_valid_f1,2))
#print()

# Тестирование:
#predictions_test = gscv.predict(features_test.text)
#cbc_test_f1 = f1_score(target_test, predictions_test)
#print('финальный F1 CatBoostClassifier =', round(cbc_test_f1,2))

F1 CatBoostClassifier = 0.74
при параметрах {'cbc__iterations': 200, 'cbc__verbose': False}

CPU times: user 32min 8s, sys: 21.8 s, total: 32min 30s
Wall time: 33min 24s


## Выводы

In [ ]:
# Создаю сводную таблицу по показателям F1, времени обучения модели и времени предсказания модели:
index = ['Логическая регрессия',
         'Дерево решений',
         'CatBoostClassifier',
        ]

data = {'F1 на обучающей выборке': [lr_train_f1,
                                    dtc_train_f1,
                                    cbc_train_f1]}

#        'F1 на валидационной выборке': [lr_valid_f1,
#                                        dtc_valid_f1,
#                                        cbc_valid_f1],
#
#        'F1 на тестовой выборке (финальный)': [lr_test_f1,
#                                               dtc_test_f1,
#                                               cbc_test_f1]}

f1_data = pd.DataFrame(data=data, index=index)

f1_data.sort_values(by='F1 на обучающей выборке', ascending=False)

,F1 на обучающей выборке
Логическая регрессия,0.769170
CatBoostClassifier,0.739465
Дерево решений,0.670016


In [ ]:
features_test = features_test.text

# Применяю обученную модель на тестовых данных
predictions = gscv.predict(features_test)

# Рассчитываю F1-меру на тестовой выборке
lr_test_f1 = f1_score(target_test, predictions)

print('F1 логистической регрессии на тестовой выборке =', round(lr_test_f1, 2))

F1 логистической регрессии на тестовой выборке = 0.76


**Вывод:**

В проекте:

1. Загрузил данные датафрейма и провел предобработку - удаление лишних данных, очистку текстов, лемматизацию
2. Обучила 3 моделей с разными гиперпараметрами и выборками
3. Определил лучшую модель, критерий отбора - показатель F1

Аутсайдером среди моделей стали модели Дерево решений и CatBoostClassifier.

Наилучшей моделью стала LogisticRegression, которая на тестировании показала F1 = 0.76. Поскольку требовалось найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75, рекомендовать могу LogisticRegression